In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/resnet_waymo.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-09 12:25:00,974 SSD INFO: Loaded configuration file configs/resnet_waymo.yaml
2020-04-09 12:25:00,976 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet'
        PRETRAINED: True
        #TODO: check whether output fetures are correct for resnet 34
        OUT_CHANNELS: [512, 1024, 1024, 1024, 2048, 2048]
        INPUT_CHANNELS: 3
    PRIORS:
        FEATURE_MAPS: [[40,30], [20, 15], [20,15], [20,15], [10,8], [1,1]]
        # strides should be image size/feature_map
        STRIDES: [[8,8], [16,16], [16,16], [16,16], [32,30], [320, 240]]
INPUT:
    IMAGE_SIZE: [320, 240]
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
SOLVER:
    MAX_ITER: 120000
    LR_STEPS: [80000, 100000]
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 5e-3
OUTPUT_DIR: 'outputs/resnet'
DATASET_DIR: "datasets"
2020-04-09 12:25:00,977 SSD INFO: Running with config:
DATASETS:
  TEST: ('waymo_val',)
  TRAIN: ('waymo_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 4
  P

In [4]:
model = start_train(cfg)

resnet
Detector initialized. Total Number of params:  28.54M
Backbone number of parameters: 25.56M
SSD Head number of parameters: 2.99M
2020-04-09 12:25:05,742 SSD.trainer INFO: No checkpoint found.
17936
Dataset loaded. Subset: train, number of images: 14348
2020-04-09 12:25:07,577 SSD.trainer INFO: Start training ...
2020-04-09 12:25:16,984 SSD.trainer INFO: iter: 000010, lr: 0.00173, total_loss: 15.374 (15.374), reg_loss: 7.436 (7.436), cls_loss: 7.939 (7.939), time: 0.940 (0.940), eta: 1 day, 7:20:01, mem: 6639M
2020-04-09 12:25:19,702 SSD.trainer INFO: iter: 000020, lr: 0.00180, total_loss: 13.014 (14.194), reg_loss: 7.082 (7.259), cls_loss: 5.932 (6.935), time: 0.272 (0.606), eta: 20:11:43, mem: 6639M
2020-04-09 12:25:22,403 SSD.trainer INFO: iter: 000030, lr: 0.00187, total_loss: 11.380 (13.256), reg_loss: 6.325 (6.948), cls_loss: 5.055 (6.308), time: 0.270 (0.494), eta: 16:27:47, mem: 6639M
2020-04-09 12:25:25,108 SSD.trainer INFO: iter: 000040, lr: 0.00193, total_loss: 10.090 

100%|██████████| 359/359 [03:36<00:00,  1.19it/s]


2020-04-09 12:31:13,125 SSD.inference INFO: mAP: 0.0303
vehicle         : 0.0909
person          : 0.0000
sign            : nan
cyclist         : 0.0000

2020-04-09 12:31:15,792 SSD.trainer INFO: iter: 000510, lr: 0.00500, total_loss: 7.416 (8.307), reg_loss: 4.629 (5.234), cls_loss: 2.787 (3.073), time: 22.288 (0.722), eta: 23:57:49, mem: 6639M
2020-04-09 12:31:18,603 SSD.trainer INFO: iter: 000520, lr: 0.00500, total_loss: 7.110 (8.284), reg_loss: 4.434 (5.219), cls_loss: 2.676 (3.065), time: 0.281 (0.713), eta: 23:40:48, mem: 6639M
2020-04-09 12:31:21,387 SSD.trainer INFO: iter: 000530, lr: 0.00500, total_loss: 7.206 (8.264), reg_loss: 4.541 (5.206), cls_loss: 2.666 (3.058), time: 0.278 (0.705), eta: 23:24:21, mem: 6639M
2020-04-09 12:31:23,943 SSD.trainer INFO: iter: 000540, lr: 0.00500, total_loss: 7.170 (8.244), reg_loss: 4.552 (5.194), cls_loss: 2.618 (3.049), time: 0.256 (0.697), eta: 23:07:39, mem: 6639M
2020-04-09 12:31:26,534 SSD.trainer INFO: iter: 000550, lr: 0.00500, tota

100%|██████████| 359/359 [02:31<00:00,  2.73it/s]


2020-04-09 12:36:14,915 SSD.inference INFO: mAP: 0.0606
vehicle         : 0.0909
person          : 0.0909
sign            : nan
cyclist         : 0.0000

2020-04-09 12:36:17,639 SSD.trainer INFO: iter: 001010, lr: 0.00500, total_loss: 7.270 (7.785), reg_loss: 4.649 (4.913), cls_loss: 2.621 (2.873), time: 15.870 (0.663), eta: 21:55:40, mem: 6639M
2020-04-09 12:36:20,305 SSD.trainer INFO: iter: 001020, lr: 0.00500, total_loss: 7.292 (7.781), reg_loss: 4.667 (4.910), cls_loss: 2.626 (2.870), time: 0.267 (0.660), eta: 21:47:50, mem: 6639M
2020-04-09 12:36:23,180 SSD.trainer INFO: iter: 001030, lr: 0.00500, total_loss: 7.007 (7.773), reg_loss: 4.403 (4.905), cls_loss: 2.604 (2.868), time: 0.288 (0.656), eta: 21:40:34, mem: 6639M
2020-04-09 12:36:26,103 SSD.trainer INFO: iter: 001040, lr: 0.00500, total_loss: 6.949 (7.765), reg_loss: 4.262 (4.899), cls_loss: 2.688 (2.866), time: 0.292 (0.652), eta: 21:33:32, mem: 6639M
2020-04-09 12:36:28,704 SSD.trainer INFO: iter: 001050, lr: 0.00500, tota

KeyboardInterrupt: 

###### 